In [ ]:
from IPython.display import clear_output

# Content

In this notebook you will finetune a transformer network taken from huggingface, using huggingface's Trainer API

you will train facebook's mbart model to create a two way chinese to urdu language translator.

The reason for chosing mbart as the source model is because it has already seen a lot of languages's data(50 in our case) during pre-training

you can find more information about mbart on [hugging face](https://huggingface.co/facebook/mbart-large-50)

In [ ]:
# %pip install gdown
# %pip install evaluate
# %pip install pandas
# %pip install sentencepiece
# %pip install accelerator
# %pip install protobuf==3.20.3
# %pip install matplotlib
# %pip install transformers datasets

# %pip install --disable-pip-version-check \
#     torch \
#     torchdata --quiet

# %pip install tqdm

clear_output()

In [22]:
%pip install datasets
%pip install transformers[torch]  # [torch] because Trainer API with torch needs accelerator installed. This takes care of that. Might need a restart of the session
%pip install gdown==4.5
%pip install evaluate
%pip install rouge_score

clear_output()

In [42]:
import math

import pandas as pd
import gdown

from datasets import load_dataset, Dataset
from transformers import GenerationConfig, TrainingArguments, Trainer, TrainerCallback
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import torch
import time
import pandas as pd
import evaluate
from tqdm import tqdm
import json

import matplotlib.pyplot as plt

import random

## Downloading the data

In [ ]:
!gdown 1ivqxeMVKDrHtjxBf3QXLAo7T9GciqNy-  # train.csv
!gdown 1Y0Ls3Rzr9MJr07fw91mFf2iSagl5CVt5  # test.csv

Downloading...
From: https://drive.google.com/uc?id=1ivqxeMVKDrHtjxBf3QXLAo7T9GciqNy-
To: /content/train.csv
100% 16.4M/16.4M [00:00<00:00, 165MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Y0Ls3Rzr9MJr07fw91mFf2iSagl5CVt5
To: /content/test.csv
100% 1.82M/1.82M [00:00<00:00, 155MB/s]


In [ ]:
random.seed(7)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
train_data = pd.read_csv('train.csv', usecols=['urdu', 'chinese'])
test_data = pd.read_csv('test.csv', usecols=['urdu', 'chinese'])

In [ ]:
train_data.head()

,urdu,chinese
0,اگرچہ وی چیٹ کے حکام کریک ڈاؤن جاری رکھتے ہیں،...,虽然微信官方不断打击，但要根治侵权‘清粉’软件恐怕并不容易。
1,کی علامات کا جلد از جلد علاج کرنے کے لیے، مریض...,为了尽快调理好内分泌失调的症状，患者需要保持好的心态。\nendocrine dyscrasia
2,زائرین کی طرف سے اکثر یہ پوچھا جاتا ہے کہ صرف ...,常有参观者问，为什么只能找到一万多个姓名？这也一直是史学研究者们的心结。
3,سترھویں سی پی سی مرکزی کمیٹی کے ساتویں مکمل اج...,这是党的十七届六中全会提出建设社会主义文化强国以来，党中央首次明确了建成文化强国的具体时间表。
4,سفر میں تبدیلی کا موقع بڑا ہے، اسی لئے چاہے آن...,旅行是未知的，所以是否买往返的联成票要注意。


In [ ]:
train_dataset = Dataset.from_pandas(train_data, split='train')
test_dataset = Dataset.from_pandas(test_data, split='test')

## Prepare the model and tokenizers

load the facebook/mbart-large-50 model and tokenizer and tokenize the dataset

try tokenizing and de-tokenizing a sentence to see if your tokenizer is working correctly

## Training the model

Fine tune the model on the training set using the trainer api

## Testing

Take a few examples from the test set and try translating them. Show the results

## Evaluating the results

Translate the test data using non-fine tuned and finetuned models

for both models, calculate the BLEU score

Show the results